# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

/home/sabau/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([25, 58, 51, 66, 70, 55, 68,  1, 11,  0,  0,  0, 30, 51, 66, 66, 75,
        1, 56, 51, 63, 59, 62, 59, 55, 69,  1, 51, 68, 55,  1, 51, 62, 62,
        1, 51, 62, 59, 61, 55, 21,  1, 55, 72, 55, 68, 75,  1, 71, 64, 58,
       51, 66, 66, 75,  1, 56, 51, 63, 59, 62, 75,  1, 59, 69,  1, 71, 64,
       58, 51, 66, 66, 75,  1, 59, 64,  1, 59, 70, 69,  1, 65, 73, 64,  0,
       73, 51, 75,  9,  0,  0, 27, 72, 55, 68, 75, 70, 58, 59, 64],
      dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

77

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [6]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[0:n_batches*characters_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        tmp = arr[:, n+1:n+1+n_steps]
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:tmp.shape[1]] = tmp
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[25 58 51 66 70 55 68  1 11  0]
 [53 61  1 58 55 68  7  1 65 56]
 [ 1 51 70  0 70 58 55  1 69 51]
 [70 58 55 68  4 69  1 68 59 57]
 [22  1 24 75  1 53 58 51 64 53]
 [51 54 55 63 65 59 69 55 62 62]
 [62 51 63 55  1 69 65 63 55 65]
 [ 1 41 55 68 57 55 75  1 31 72]
 [58 55 63  1 53 65 63 66 62 55]
 [58  1 51  1 62 55 70 70 55 68]]

y
 [[58 51 66 70 55 68  1 11  0  0]
 [61  1 58 55 68  7  1 65 56  1]
 [51 70  0 70 58 55  1 69 51 63]
 [58 55 68  4 69  1 68 59 57 58]
 [ 1 24 75  1 53 58 51 64 53 55]
 [54 55 63 65 59 69 55 62 62 55]
 [51 63 55  1 69 65 63 55 65 64]
 [41 55 68 57 55 75  1 31 72 51]
 [55 63  1 53 65 63 66 62 55 70]
 [ 1 51  1 62 55 70 70 55 68  9]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [9]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='tagrets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    def lstm_cell(lstm_size, keep_prob):
        ### Build the LSTM Cell
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

        # Add dropout to the cell outputs
        return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='out')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped))
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [19]:
batch_size = 64         # Sequences per batch
num_steps = 10          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [20]:
epochs = 20
# Print losses every N interations
print_every_n = 50

# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 50...  Training loss: 3.1828...  0.2634 sec/batch
Epoch: 1/20...  Training Step: 100...  Training loss: 3.1322...  0.3007 sec/batch
Epoch: 1/20...  Training Step: 150...  Training loss: 2.7386...  0.3998 sec/batch
Epoch: 1/20...  Training Step: 200...  Training loss: 2.5834...  0.2375 sec/batch
Epoch: 1/20...  Training Step: 250...  Training loss: 2.4582...  0.2525 sec/batch
Epoch: 1/20...  Training Step: 300...  Training loss: 2.3518...  0.2317 sec/batch
Epoch: 1/20...  Training Step: 350...  Training loss: 2.2316...  0.2533 sec/batch
Epoch: 1/20...  Training Step: 400...  Training loss: 2.2252...  0.2844 sec/batch
Epoch: 1/20...  Training Step: 450...  Training loss: 2.2034...  0.2726 sec/batch
Epoch: 1/20...  Training Step: 500...  Training loss: 2.0437...  0.2557 sec/batch
Epoch: 1/20...  Training Step: 550...  Training loss: 2.1122...  0.2348 sec/batch
Epoch: 1/20...  Training Step: 600...  Training loss: 2.0237...  0.3033 sec/batch
Epoch: 1/20...  T

Epoch: 2/20...  Training Step: 5000...  Training loss: 1.3620...  0.3004 sec/batch
Epoch: 2/20...  Training Step: 5050...  Training loss: 1.4266...  0.2915 sec/batch
Epoch: 2/20...  Training Step: 5100...  Training loss: 1.4435...  0.2591 sec/batch
Epoch: 2/20...  Training Step: 5150...  Training loss: 1.4040...  0.2414 sec/batch
Epoch: 2/20...  Training Step: 5200...  Training loss: 1.4980...  0.2659 sec/batch
Epoch: 2/20...  Training Step: 5250...  Training loss: 1.2731...  0.4161 sec/batch
Epoch: 2/20...  Training Step: 5300...  Training loss: 1.3166...  0.2428 sec/batch
Epoch: 2/20...  Training Step: 5350...  Training loss: 1.3873...  0.3463 sec/batch
Epoch: 2/20...  Training Step: 5400...  Training loss: 1.3156...  0.2570 sec/batch
Epoch: 2/20...  Training Step: 5450...  Training loss: 1.3967...  0.2744 sec/batch
Epoch: 2/20...  Training Step: 5500...  Training loss: 1.3883...  0.2724 sec/batch
Epoch: 2/20...  Training Step: 5550...  Training loss: 1.4237...  0.2541 sec/batch
Epoc

Epoch: 4/20...  Training Step: 9950...  Training loss: 1.3489...  0.2314 sec/batch
Epoch: 4/20...  Training Step: 10000...  Training loss: 1.2610...  0.2531 sec/batch
Epoch: 4/20...  Training Step: 10050...  Training loss: 1.2778...  0.2322 sec/batch
Epoch: 4/20...  Training Step: 10100...  Training loss: 1.2562...  0.2551 sec/batch
Epoch: 4/20...  Training Step: 10150...  Training loss: 1.1557...  0.2321 sec/batch
Epoch: 4/20...  Training Step: 10200...  Training loss: 1.2628...  0.2532 sec/batch
Epoch: 4/20...  Training Step: 10250...  Training loss: 1.3596...  0.4502 sec/batch
Epoch: 4/20...  Training Step: 10300...  Training loss: 1.2962...  0.2498 sec/batch
Epoch: 4/20...  Training Step: 10350...  Training loss: 1.3328...  0.2396 sec/batch
Epoch: 4/20...  Training Step: 10400...  Training loss: 1.3340...  0.2386 sec/batch
Epoch: 4/20...  Training Step: 10450...  Training loss: 1.2647...  0.2507 sec/batch
Epoch: 4/20...  Training Step: 10500...  Training loss: 1.1338...  0.2360 sec

Epoch: 5/20...  Training Step: 14850...  Training loss: 1.2704...  0.2290 sec/batch
Epoch: 5/20...  Training Step: 14900...  Training loss: 1.1926...  0.2486 sec/batch
Epoch: 5/20...  Training Step: 14950...  Training loss: 1.3180...  0.4488 sec/batch
Epoch: 5/20...  Training Step: 15000...  Training loss: 1.2964...  0.2503 sec/batch
Epoch: 5/20...  Training Step: 15050...  Training loss: 1.2347...  0.4488 sec/batch
Epoch: 5/20...  Training Step: 15100...  Training loss: 1.1470...  0.2532 sec/batch
Epoch: 5/20...  Training Step: 15150...  Training loss: 1.1852...  0.2479 sec/batch
Epoch: 5/20...  Training Step: 15200...  Training loss: 1.2575...  0.3750 sec/batch
Epoch: 5/20...  Training Step: 15250...  Training loss: 1.0435...  0.2527 sec/batch
Epoch: 5/20...  Training Step: 15300...  Training loss: 1.1762...  0.3712 sec/batch
Epoch: 5/20...  Training Step: 15350...  Training loss: 1.1730...  0.2511 sec/batch
Epoch: 6/20...  Training Step: 15400...  Training loss: 1.1637...  0.2314 se

Epoch: 7/20...  Training Step: 19750...  Training loss: 1.1402...  0.4489 sec/batch
Epoch: 7/20...  Training Step: 19800...  Training loss: 1.2034...  0.2316 sec/batch
Epoch: 7/20...  Training Step: 19850...  Training loss: 1.2144...  0.2510 sec/batch
Epoch: 7/20...  Training Step: 19900...  Training loss: 1.2155...  0.2328 sec/batch
Epoch: 7/20...  Training Step: 19950...  Training loss: 1.1935...  0.2529 sec/batch
Epoch: 7/20...  Training Step: 20000...  Training loss: 1.1613...  0.2351 sec/batch
Epoch: 7/20...  Training Step: 20050...  Training loss: 1.1942...  0.2537 sec/batch
Epoch: 7/20...  Training Step: 20100...  Training loss: 1.2728...  0.2346 sec/batch
Epoch: 7/20...  Training Step: 20150...  Training loss: 1.1474...  0.2504 sec/batch
Epoch: 7/20...  Training Step: 20200...  Training loss: 1.1803...  0.4488 sec/batch
Epoch: 7/20...  Training Step: 20250...  Training loss: 1.2359...  0.2523 sec/batch
Epoch: 7/20...  Training Step: 20300...  Training loss: 1.1222...  0.2318 se

...

Epoch: 13/20...  Training Step: 39250...  Training loss: 1.1237...  0.2503 sec/batch
Epoch: 13/20...  Training Step: 39300...  Training loss: 1.1619...  0.3697 sec/batch
Epoch: 13/20...  Training Step: 39350...  Training loss: 1.1769...  0.2504 sec/batch
Epoch: 13/20...  Training Step: 39400...  Training loss: 1.2141...  0.4282 sec/batch
Epoch: 13/20...  Training Step: 39450...  Training loss: 1.0678...  0.2513 sec/batch
Epoch: 13/20...  Training Step: 39500...  Training loss: 1.1817...  0.2328 sec/batch
Epoch: 13/20...  Training Step: 39550...  Training loss: 1.1568...  0.2515 sec/batch
Epoch: 13/20...  Training Step: 39600...  Training loss: 1.1382...  0.2317 sec/batch
Epoch: 13/20...  Training Step: 39650...  Training loss: 1.1714...  0.2519 sec/batch
Epoch: 13/20...  Training Step: 39700...  Training loss: 1.0864...  0.4517 sec/batch
Epoch: 13/20...  Training Step: 39750...  Training loss: 1.1259...  0.2497 sec/batch
Epoch: 13/20...  Training Step: 39800...  Training loss: 1.1504..

Epoch: 15/20...  Training Step: 44100...  Training loss: 1.1215...  0.2329 sec/batch
Epoch: 15/20...  Training Step: 44150...  Training loss: 1.0841...  0.3720 sec/batch
Epoch: 15/20...  Training Step: 44200...  Training loss: 1.1243...  0.2535 sec/batch
Epoch: 15/20...  Training Step: 44250...  Training loss: 1.1517...  0.3711 sec/batch
Epoch: 15/20...  Training Step: 44300...  Training loss: 1.1049...  0.2602 sec/batch
Epoch: 15/20...  Training Step: 44350...  Training loss: 1.1953...  0.2328 sec/batch
Epoch: 15/20...  Training Step: 44400...  Training loss: 1.1200...  0.2503 sec/batch
Epoch: 15/20...  Training Step: 44450...  Training loss: 1.1800...  0.2304 sec/batch
Epoch: 15/20...  Training Step: 44500...  Training loss: 1.1813...  0.2483 sec/batch
Epoch: 15/20...  Training Step: 44550...  Training loss: 1.0869...  0.2318 sec/batch
Epoch: 15/20...  Training Step: 44600...  Training loss: 1.1260...  0.2476 sec/batch
Epoch: 15/20...  Training Step: 44650...  Training loss: 0.9500..

Epoch: 16/20...  Training Step: 48950...  Training loss: 1.0958...  0.2305 sec/batch
Epoch: 16/20...  Training Step: 49000...  Training loss: 1.0859...  0.2472 sec/batch
Epoch: 16/20...  Training Step: 49050...  Training loss: 1.1525...  0.4466 sec/batch
Epoch: 16/20...  Training Step: 49100...  Training loss: 1.1528...  0.2497 sec/batch
Epoch: 16/20...  Training Step: 49150...  Training loss: 1.1596...  0.2299 sec/batch
Epoch: 17/20...  Training Step: 49200...  Training loss: 1.1358...  0.2503 sec/batch
Epoch: 17/20...  Training Step: 49250...  Training loss: 1.0934...  0.4499 sec/batch
Epoch: 17/20...  Training Step: 49300...  Training loss: 1.1289...  0.2486 sec/batch
Epoch: 17/20...  Training Step: 49350...  Training loss: 1.2140...  0.2305 sec/batch
Epoch: 17/20...  Training Step: 49400...  Training loss: 1.0901...  0.2499 sec/batch
Epoch: 17/20...  Training Step: 49450...  Training loss: 1.1187...  0.2310 sec/batch
Epoch: 17/20...  Training Step: 49500...  Training loss: 1.1110..

Epoch: 18/20...  Training Step: 53800...  Training loss: 1.0993...  0.2346 sec/batch
Epoch: 18/20...  Training Step: 53850...  Training loss: 1.1176...  0.2517 sec/batch
Epoch: 18/20...  Training Step: 53900...  Training loss: 1.1134...  0.2297 sec/batch
Epoch: 18/20...  Training Step: 53950...  Training loss: 1.1091...  0.2535 sec/batch
Epoch: 18/20...  Training Step: 54000...  Training loss: 1.0411...  0.2334 sec/batch
Epoch: 18/20...  Training Step: 54050...  Training loss: 1.0770...  0.2518 sec/batch
Epoch: 18/20...  Training Step: 54100...  Training loss: 1.0877...  0.4490 sec/batch
Epoch: 18/20...  Training Step: 54150...  Training loss: 1.0685...  0.2307 sec/batch
Epoch: 18/20...  Training Step: 54200...  Training loss: 1.1248...  0.2516 sec/batch
Epoch: 18/20...  Training Step: 54250...  Training loss: 1.1098...  0.2402 sec/batch
Epoch: 18/20...  Training Step: 54300...  Training loss: 1.1650...  0.2501 sec/batch
Epoch: 18/20...  Training Step: 54350...  Training loss: 1.1103..

Epoch: 20/20...  Training Step: 58650...  Training loss: 1.2009...  0.3686 sec/batch
Epoch: 20/20...  Training Step: 58700...  Training loss: 1.0456...  0.2553 sec/batch
Epoch: 20/20...  Training Step: 58750...  Training loss: 1.0462...  0.2320 sec/batch
Epoch: 20/20...  Training Step: 58800...  Training loss: 1.0624...  0.2486 sec/batch
Epoch: 20/20...  Training Step: 58850...  Training loss: 1.1745...  0.2301 sec/batch
Epoch: 20/20...  Training Step: 58900...  Training loss: 1.0976...  0.2507 sec/batch
Epoch: 20/20...  Training Step: 58950...  Training loss: 1.1227...  0.2288 sec/batch
Epoch: 20/20...  Training Step: 59000...  Training loss: 1.2139...  0.2502 sec/batch
Epoch: 20/20...  Training Step: 59050...  Training loss: 1.1499...  0.3694 sec/batch
Epoch: 20/20...  Training Step: 59100...  Training loss: 1.1517...  0.2508 sec/batch
Epoch: 20/20...  Training Step: 59150...  Training loss: 1.1785...  0.3726 sec/batch
Epoch: 20/20...  Training Step: 59200...  Training loss: 1.0879..

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [21]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i61440_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i41800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i42000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i42200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i42400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i42600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i42800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i43000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i43200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i43400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i43600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i43800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i44000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i44200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i44400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i44600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i44800_l512.ckpt"
al

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [22]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [23]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [24]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i61440_l512.ckpt'

In [25]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i61440_l512.ckpt
Farenst in the presence of the prince to start over the consciousness of his wife. That's any
dine at the thought of a sticky part of the
cause of all men would have seemed to see her at the same time if he did a strange court to him.

"You must be always being such
a friend, talking to
a
crime, which she was, in her husband, the position is awkward to her in her house. The wint was nothing. To my selves. I'm always to say if you were since he wants to be meadow," said the ludicrous forehead, and her flighted head of his face.

"And tell me about a single, something sees, but it is, take the class of the service," she said, and she had stung it, saw the most sended face, and the matter, as he had always been a lady or
the compassion of whom they were not to say something that shook his hands, and saw his wife, all the particularly could not have been seeking so much for him and winder.

"I don't know. And here he ha

In [26]:
checkpoint = 'checkpoints/i41800_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i41800_l512.ckpt
Farenish was sending to her, so he could not be dreamed over. He saw himself, he clutched his shirt.

"Yes, it can have been to be able to say, and he didn't see yourself."

"No; we'll be able to be
angry, and there are assembled to him. How called them and so much to see me," said Sergey Ivanovitch, smiling, and she saw that in some man had been talked about her, than the children he had been studiously
as to all her
conversation.

His head was an agreed to
think of her, he could see, and his brother, though he was
saying.

"Then they're all that tongue that will not be a minute; but if they're there was in all this."

Alexey Alexandrovitch, she went up to her, softening a short clook of thrown
off of what had been done the
same thing, began to said that three distinction was said, and still hoped
he saw the men, and all the country
could not believe that in which how she had never concealed that
he
did not see tha

In [27]:
checkpoint = 'checkpoints/i50000_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i50000_l512.ckpt
Fareney band, well, and that's the center," said Levin.

"I'm a celebrated peasant will have something, I've nothing to say to her son and stand, but then anyone who had been that he saw that the conversation was out. It appeared, but a fite infacturily opened his head
in the last things. They were assented and couried, but they were not louding it to his wife in her eyes, and she was at the sight of him, and she with her head sat down the colonel. And she shook himself. All
these moment he fell to to be pulled out the streets. Alexey Alexandrovitch had seen her, and he had never been doing the business, which
had been spoken, and
to be so as not a sense of that, and so that the children, who was it that his brother
Nikolay had taken a side of the books.

"I can be asked, though it's not an action," he said,
and was standing up the same plants, transparent, to
shot, was now thinking of the contempt and her hands on 

In [28]:
checkpoint = 'checkpoints/i60000_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i60000_l512.ckpt
Farenity's and sincing my
heart."

"I don't want to get on one another."

And
in well-kepred to a child, but she felt the
manight and that they should not be alled, as though
it were, took off her face in the morning about her. The colonel, she. "I shall be done in anything," she whispered.

"What do you do it?" said the conversation, and he smiled at his head, and at the stillness of the second.

"I want to to suppress the caping service." Alexey
Alexandrovitch was at home. The most amusable she was not a life
alone, and
all this
would be a secret of the
country, and
always began to be able to say about him, and her
heart the court had the cross of his childhood and her
children, and the peasant was stared at something, and took off the window in his creet and cheese. "I shall see the
thought and was
seeking," he thought, when he had been all to a short small
coat, and a silence, the presence, and his beards of
the